In [2]:
from influence.hessians import *
from influence.inception_v3 import *
from influence.dataset import DataSet
from influence.inceptionModel import *

import tensorflow as tf

import pandas as pd

#from keras.preprocessing import image
#from keras import backend as K

Using Theano backend.


In [3]:
import os
from cleverhans.attacks import FastGradientMethod
from io import BytesIO
import IPython.display
import numpy as np
import pandas as pd
from PIL import Image
from scipy.misc import imread
from scipy.misc import imsave
import tensorflow as tf
from tensorflow.contrib.slim.nets import inception

from tensorflow.contrib.learn.python.learn.datasets import base

In [8]:
x_input = tf.placeholder(tf.float32, shape=[16, image_height, image_width, 3])

target_class_input = tf.placeholder(tf.int32, shape=[16])
one_hot_target_class = tf.one_hot(target_class_input, num_classes)

with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(
            x_input, num_classes=num_classes, is_training=False, reuse = True)

    
cross_entropy = tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                    logits,
                                                    label_smoothing=0.1,
                                                    weights=1.0)

cross_entropy += tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                     end_points['AuxLogits'],
                                                     label_smoothing=0.1,
                                                     weights=0.4)

weights = []
for _ in range(16):
    weights.append(tf.placeholder(tf.float32, shape=[10, 10, 3]))
    
v = []
for _ in range(16):
    v.append(tf.ones(shape = [10, 10, 3]))

hess_vector_product = hessian_vector_product(cross_entropy, weights, v)

with tf.Session() as sess:
    #sess.run(init_op)
    feed_dict = {x_input: np.random.random( [ 16, 299, 299, 3 ] ), 
                 one_hot_target_class: np.random.random( [ 16,  1001] )}
    #print(sess.run(loss, feed_dict))
    print(sess.run(hess_vector_product, feed_dict))

[array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,

## what's below is still under testing

In [4]:
slim = tf.contrib.slim
tensorflow_master = ""
checkpoint_path   = "./input/inception-v3/inception_v3.ckpt"
max_epsilon       = 16.0
image_width       = 299
image_height      = 299
batch_size        = 16

eps = 2.0 * max_epsilon / 255.0
batch_shape = [batch_size, image_height, image_width, 3]
num_classes = 1001

In [5]:
img_side = 299
num_channels = 3
 
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [100, 1000]

weight_decay = 0.001

num_classes = 2
max_lbfgs_iter = 1000

num_train_ex_per_class = 900
num_test_ex_per_class = 300
batch_size = 16

In [114]:
#base.Datasets(train = Train)

def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

img = image.load_img('0aebe24fc257286e.png', target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

x = preprocess_input(x)

In [115]:
df = pd.read_csv('images.csv')

def get_sorted_files(Directory):
    filenamelist = []
    for root, dirs, files in os.walk(Directory):
        for name in files:
            fullname = os.path.join(root, name)
            filenamelist.append(fullname)
    return sorted(filenamelist)

In [116]:
train_X = []

for filename in get_sorted_files('./dataset/images'):
    img = image.load_img(filename, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    train_X.append(preprocess_input(x))
    
train_y = list((df['TrueLabel']>499).astype(int))

In [117]:
datasets = DataSet(np.array(train_X), np.array(train_y))

In [118]:
full_model = BinaryInceptionModel(
        img_side=img_side,
        num_channels=num_channels,
        weight_decay=weight_decay,
        num_classes=num_classes, 
        batch_size=batch_size,
        data_sets=data_sets,
        initial_learning_rate=initial_learning_rate,
        keep_probs=keep_probs,
        decay_epochs=decay_epochs,
        mini_batch=True,
        train_dir='output',
        log_dir='log',
        model_name = 'v1')

AttributeError: 'module' object has no attribute 'set_session'

In [6]:
class InceptionModel(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.built = False

    def __call__(self, x_input):
        """Constructs model and return probabilities for given input."""
        reuse = True if self.built else None
        with slim.arg_scope(inception.inception_v3_arg_scope()):
            _, end_points = inception.inception_v3(
                            x_input, num_classes=self.num_classes, is_training=False,
                            reuse=reuse)
        self.built = True
        output = end_points['Predictions']
        probs = output.op.inputs[0]
        return probs

In [7]:
def load_images(input_dir, batch_shape):
    images = np.zeros(batch_shape)
    filenames = []
    idx = 0
    batch_size = batch_shape[0]
    for filepath in sorted(tf.gfile.Glob(os.path.join(input_dir, '*.png'))):
        with tf.gfile.Open(filepath, "rb") as f:
            images[idx, :, :, :] = imread(f, mode='RGB').astype(np.float)*2.0/255.0 - 1.0
        filenames.append(os.path.basename(filepath))
        idx += 1
        if idx == batch_size:
            yield filenames, images
            filenames = []
            images = np.zeros(batch_shape)
            idx = 0
    if idx > 0:
        yield filenames, images

In [8]:
input_dir  = "./input/images/"

image_iterator = load_images(input_dir, batch_shape)

# get first batch of images
filenames, images = next(image_iterator)

In [15]:
np.random.random([10, 10,10,10])

array([[[[ 0.18950933,  0.39552469,  0.45785469, ...,  0.4841708 ,
           0.31152802,  0.07108792],
         [ 0.83118851,  0.17725421,  0.96623389, ...,  0.37236339,
           0.22520074,  0.62836488],
         [ 0.4257087 ,  0.60293347,  0.93292023, ...,  0.45404316,
           0.42497194,  0.35537391],
         ..., 
         [ 0.77003797,  0.68453843,  0.27242565, ...,  0.51793155,
           0.68566976,  0.57857804],
         [ 0.35471112,  0.19252271,  0.29676724, ...,  0.11901539,
           0.67264678,  0.28865729],
         [ 0.53621917,  0.7896549 ,  0.05815355, ...,  0.78288123,
           0.88207968,  0.65735433]],

        [[ 0.58313149,  0.85485505,  0.35511443, ...,  0.40983709,
           0.1901042 ,  0.87168706],
         [ 0.16761064,  0.98571668,  0.20102237, ...,  0.98633145,
           0.00135152,  0.53023393],
         [ 0.62497967,  0.09654504,  0.41092591, ...,  0.80401943,
           0.19504864,  0.053704  ],
         ..., 
         [ 0.34105007,  0.804830

In [31]:
num_classes

2

In [33]:
x_input = tf.placeholder(tf.float32, shape=[16, image_height, image_width, 3])

target_class_input = tf.placeholder(tf.int32, shape=[16])
one_hot_target_class = tf.one_hot(target_class_input, num_classes)

with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(
            x_input, num_classes=num_classes, is_training=False, reuse = True)

    
cross_entropy = tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                    logits,
                                                    label_smoothing=0.1,
                                                    weights=1.0)

cross_entropy += tf.losses.softmax_cross_entropy(one_hot_target_class,
                                                     end_points['AuxLogits'],
                                                     label_smoothing=0.1,
                                                     weights=0.4)

weights = []
for _ in range(16):
    weights.append(tf.placeholder(tf.float32, shape=[10, 10, 3]))
    
v = []
for _ in range(16):
    v.append(tf.ones(shape = [10, 10, 3]))

hess = hessian_vector_product(cross_entropy, weights, v)

with tf.Session() as sess:
    #sess.run(init_op)
    feed_dict = {x_input: np.random.random( [ 16, 299, 299, 3 ] ), 
                 one_hot_target_class: np.random.random( [ 16,  2] )}
    #print(sess.run(loss, feed_dict))
    print(sess.run(hess, feed_dict))

[array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]],

       [[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,

In [29]:
np.random.random( [ 16, 299, 299, 3 ] )

array([[[[ 0.32420591,  0.25919855,  0.21779949],
         [ 0.70574964,  0.67936129,  0.99555705],
         [ 0.94167858,  0.44187256,  0.56246596],
         ..., 
         [ 0.73869335,  0.27136104,  0.16723534],
         [ 0.14464947,  0.54178791,  0.95416815],
         [ 0.77783759,  0.74100385,  0.25127388]],

        [[ 0.41056585,  0.86969952,  0.63850564],
         [ 0.38614738,  0.47447972,  0.2900787 ],
         [ 0.95777031,  0.5572539 ,  0.13181849],
         ..., 
         [ 0.95595173,  0.40337393,  0.79588919],
         [ 0.2423526 ,  0.04277214,  0.35949957],
         [ 0.35222423,  0.95136711,  0.83353129]],

        [[ 0.95584457,  0.51274988,  0.80343297],
         [ 0.45139142,  0.82642339,  0.33335765],
         [ 0.439845  ,  0.73498051,  0.32894714],
         ..., 
         [ 0.41077881,  0.42683621,  0.99789084],
         [ 0.4210893 ,  0.3801408 ,  0.32923082],
         [ 0.83532325,  0.90043703,  0.4663367 ]],

        ..., 
        [[ 0.45082028,  0.45213206,

In [27]:
feed_dict = {x_input: np.random.random( [ 16, 299, 299, 3 ] ), 
                 y_target: np.random.random( [ 16, 1 ] )}

TypeError: unhashable type: 'list'

In [56]:
tf.ones(
    shape = [image_height, image_width, 3]
)

<tf.Tensor 'ones:0' shape=(299, 299, 3) dtype=float32>

In [28]:
with tf.Session() as sess:
    ev = tf.Tensor.eval(hessian_vector_product([0,1], [2,3,4], [100,2,1])[0], session=sess)

In [30]:
with tf.Session() as sess:
    ev = tf.Tensor.eval(tf.gradients([0], [1,2]), session=sess)

TypeError: unbound method eval() must be called with Tensor instance as first argument (got list instance instead)

In [39]:
xs = tf.Variable([1,2,3])
ys = tf.Variable([1.2])

tf.gradients( ys, xs )

[None]

In [37]:
tf.Variable([1,2,3])

<tf.Variable 'Variable:0' shape=(3,) dtype=int32_ref>

In [136]:
hessian_vector_product([1,0], [[2,3,5,1,2],[2,3,4,12]], [[3,1,2,1,2],[5,2,6,32]])[0].eval()

array([0, 0, 0, 0, 0], dtype=int32)

In [123]:
inception.inception_v3_arg_scope()

{'<function convolution at 0x116d57aa0>': {'activation_fn': <function tensorflow.python.ops.gen_nn_ops.relu>,
  'normalizer_fn': <function tensorflow.contrib.framework.python.ops.arg_scope.batch_norm>,
  'normalizer_params': {'decay': 0.9997,
   'epsilon': 0.001,
   'updates_collections': 'update_ops',
   'variables_collections': {'beta': None,
    'gamma': None,
    'moving_mean': ['moving_vars'],
    'moving_variance': ['moving_vars']}},
  'weights_initializer': <tensorflow.python.ops.init_ops.TruncatedNormal at 0x11c8134d0>,
  'weights_regularizer': <function tensorflow.contrib.layers.python.layers.regularizers.l2>},
 '<function fully_connected at 0x116cf9668>': {'weights_regularizer': <function tensorflow.contrib.layers.python.layers.regularizers.l2>}}

In [137]:
batch_shape = [100, 1000, 1000, 3]

x_input = tf.placeholder(tf.float32, shape=batch_shape)

In [141]:
tf.get_default_graph().get_tensor_by_name()

In [142]:
InceptionV3(include_top=False, weights='imagenet', input_tensor=reshaped_input)

NameError: name 'InceptionV3' is not defined

In [86]:
tf.logging.set_verbosity(tf.logging.INFO)

with tf.Graph().as_default():
    x_input = tf.placeholder(tf.float32, shape=batch_shape)
    model = InceptionModel(num_classes)

    fgsm  = FastGradientMethod(model)
    x_adv = fgsm.generate(x_input, eps=eps, clip_min=-1., clip_max=1.)

    saver = tf.train.Saver(slim.get_model_variables())
    session_creator = tf.train.ChiefSessionCreator(
                      scaffold=tf.train.Scaffold(saver=saver),
                      checkpoint_filename_with_path=checkpoint_path,
                      master=tensorflow_master)

    with tf.train.MonitoredSession(session_creator=session_creator) as sess:
        nontargeted_images = sess.run(x_adv, feed_dict={x_input: images})

print("The original image is on the left, and the nontargeted adversarial image is on the right. They look very similar, don't they? It's very clear both are gondolas")
show_image(np.concatenate([images[1], nontargeted_images[1]], axis=1))

/Users/tianyuwang/anaconda/lib/python2.7/site-packages/cleverhans-1.0.0-py2.7.egg/cleverhans/attacks.py:41: UserWarning: CleverHans support for supplying a callable instead of an instance of the cleverhans.model.Model class is deprecated and will be dropped on 2018-01-11.
  warnings.warn("CleverHans support for supplying a callable"


INFO:tensorflow:Restoring parameters from ./input/inception-v3/inception_v3.ckpt


NameError: name 'images' is not defined

In [57]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [58]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [59]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [60]:
sess.run(tf.global_variables_initializer())

In [61]:
y = tf.matmul(x,W) + b

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [62]:
for _ in range(1000):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [70]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9193


In [64]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [65]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [73]:
W = tf.Variable([3,2,4])

In [150]:
inception.inception_v3.get

<function tensorflow.contrib.slim.python.slim.nets.inception_v3.inception_v3>